In [1]:
import pandas as pd
import sentencepiece as spm
# Replace 'your_file.parquet' with the path to your Parquet file
df = pd.read_parquet('0000.parquet')


In [2]:
df.head()

,answers,passages,query,query_id,query_type,wellFormedAnswers
0,[Yes],"{'is_selected': [0, 0, 1, 0, 0, 0, 0], 'passag...",does human hair stop squirrels,0,description,[]
1,[Fossil fuels are basically the remains of ani...,"{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0], '...",what are the benefits of fossil fuels,1,description,[]
2,[The apothem of a regular polygon is a line se...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0], '...",what is a apothem,2,description,[]
3,[$45 to $210. 2],"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0], '...",average cost for custom canopy,3,numeric,[]
4,[It is the collection of physical elements tha...,"{'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]...",what is a hardware in a computer,4,description,[]


In [3]:
import json

def extract_passages_and_ground_truth(data):
    # Check if data is a string and parse it as JSON; otherwise, use it directly if it's a dictionary
    if isinstance(data, str):
        parsed_json = json.loads(data)
    elif isinstance(data, dict):
        parsed_json = data
    else:
        raise ValueError("Input must be a JSON string or a dictionary.")
    
    # Extract passages and ground truth labels assuming the data is now a dictionary
    passages = parsed_json['passage_text']
    ground_truth_labels = parsed_json['is_selected']
    
    # Split each passage into paragraphs, clean them, and pair them with ground truth labels
    cleaned_passages_with_labels = []
    for passage, label in zip(passages, ground_truth_labels):
        paragraphs = passage.split('", "')
        cleaned_paragraphs = [paragraph.strip() for paragraph in paragraphs]
        for paragraph in cleaned_paragraphs:
            cleaned_passages_with_labels.append((paragraph, label))
    
    return cleaned_passages_with_labels

# Assuming 'data' is your JSON or dictionary input
# This will now create a column with tuples of (cleaned_paragraph, is_selected)
df['cleaned_paragraphs_with_labels'] = df['passages'].apply(extract_passages_and_ground_truth)

# Now you can iterate over this new column to get both the text and the ground truth
for index, (cleaned_paragraph, is_selected) in enumerate(df['cleaned_paragraphs_with_labels'][1]):
    print(f"Document {index}: Selected {is_selected} | Text: {cleaned_paragraph}")


Document 0: Selected 0 | Text: The biggest advantage of using fossil fuels is that they can be easily stored and transported from one place to another. Large reserves of coal are therefore taken from the coal mines to the industries which are acres away from the mines. The petroleum is also taken to too far off power stations to produce energy. Fossil fuels are the highest producers of calorific value in terms of energy. This is also one of the reasons why they are still preferred over the renewable sources of energy or the alternative source
Document 1: Selected 1 | Text: Benefits of fossil fuels. Fossil fuels are basically the remains of animals and plants and these are good energy resources. The three main fossil fuels are natural gas, oil, and coal. Fossil fuels are low in cost and are very important resources for our economy. Fossil fuels are used to generate electricity used as fuels for transportation.
Document 2: Selected 0 | Text: Fossil fuels are energy resources that come fr

In [4]:
def extract_passages_and_labels(data):
    if isinstance(data, str):
        parsed_json = json.loads(data)
    elif isinstance(data, dict):
        parsed_json = data
    else:
        raise ValueError("Input must be a JSON string or a dictionary.")
    
    # Extract the relevance labels
    is_selected = parsed_json.get('is_selected', [])
    relevant_indexes = [index for index, value in enumerate(is_selected) if value == 1]
    
    return relevant_indexes
df['cleaned_passages_with_labels'] = df['passages'].apply(extract_passages_and_labels)
#df['cleaned_passages_with_labels'][14]

In [5]:
empty_or_invalid_entries = [i for i, docs in enumerate(df['cleaned_passages_with_labels']) if not isinstance(docs, list) or not docs]
print(f"Indices with empty or invalid entries: {empty_or_invalid_entries}")

Indices with empty or invalid entries: [14, 32, 40, 127, 133, 147, 176, 201, 218, 223, 242, 256, 311, 312, 333, 344, 350, 356, 385, 437, 444, 567, 577, 590, 644, 646, 695, 697, 717, 861, 879, 882, 925, 929, 1014, 1033, 1057, 1204, 1226, 1259, 1283, 1307, 1326, 1333, 1341, 1368, 1381, 1411, 1441, 1449, 1452, 1468, 1487, 1635, 1650, 1699, 1750, 1751, 1753, 1770, 1812, 1905, 1916, 1972, 1981, 2019, 2047, 2056, 2063, 2073, 2083, 2112, 2125, 2127, 2142, 2168, 2243, 2253, 2257, 2314, 2326, 2330, 2433, 2451, 2508, 2547, 2590, 2704, 2723, 2817, 2818, 2891, 2946, 2960, 3063, 3075, 3119, 3123, 3140, 3157, 3190, 3243, 3244, 3264, 3324, 3344, 3345, 3348, 3357, 3392, 3412, 3421, 3462, 3478, 3484, 3494, 3496, 3515, 3558, 3602, 3613, 3637, 3646, 3699, 3716, 3733, 3817, 3826, 3858, 3859, 3882, 3886, 3897, 3952, 4022, 4036, 4061, 4070, 4135, 4143, 4176, 4193, 4315, 4351, 4419, 4420, 4534, 4548, 4571, 4593, 4599, 4652, 4668, 4681, 4706, 4707, 4712, 4759, 4778, 4873, 4897, 4919, 4937, 4951, 4962, 5007, 5

In [6]:


import json

def extract_passages(data):
    # Check if data is a string and parse it as JSON; otherwise, use it directly if it's a dictionary
    if isinstance(data, str):
        parsed_json = json.loads(data)
    elif isinstance(data, dict):
        parsed_json = data
    else:
        raise ValueError("Input must be a JSON string or a dictionary.")
    
    # Extract passages assuming the data is now a dictionary
    passages = parsed_json['passage_text']
    
    # Split each passage into paragraphs, clean them, and return them as a list
    cleaned_passages = []
    for passage in passages:
        paragraphs = passage.split('", "')
        cleaned_paragraphs = [paragraph.strip() for paragraph in paragraphs]
        cleaned_passages.extend(cleaned_paragraphs)
    
    return cleaned_passages

# Assuming 'data' is your JSON or dictionary input
df['cleaned_paragraphs'] = df['passages'].apply(extract_passages)

# 'cleaned_paragraphs' in the DataFrame now contains lists of paragraphs

# To print each paragraph with a pause in between:
#for index, paragraphs in enumerate(df['cleaned_paragraphs']):
    #for paragraph in paragraphs:
        #print(f"Passage {index+1}, Paragraph:\n{paragraph}\n")
        #input("Press Enter to continue to the next paragraph...")



In [ ]:
for index, item in enumerate(df['query']):
    print(f"Item {index+1}:\n{item}\n")
    # You can use input() to pause after each item, or remove it to print continuously
    input("Press Enter to continue to the next item...")

In [7]:
import tokenizer as tknz
sp = spm.SentencePieceProcessor()
sp.load('m.model')
tokenizer = tknz.Tokenizer('0000.parquet')

In [8]:
max_length = 0
for passage_list in df['cleaned_paragraphs']:
    for paragraph in passage_list:
        # Assume `tokenizer.encode` returns a list of tokens.
        encoded_paragraph = tokenizer.encode(paragraph)
        max_length = max(max_length, len(encoded_paragraph))

In [9]:
print(max_length)

368


In [10]:
def encode_and_pad(text, max_length, pad_token_id, sp):
    encoded_text = tokenizer.encode(text)
    padded_text = encoded_text + [pad_token_id] * (max_length - len(encoded_text)),  # Pad at the end
        
    return padded_text

In [11]:

pad_token_id = sp.PieceToId('<pad>')
max_length = 368

#padded_passages =encode_and_pad(paragraph, max_length, pad_token_id, sp) for paragraph in sublist for sublist in df['tokenized_passages']
                      

#padded_queries = df['tokenized_queries'].apply(encode_and_pad(query, max_length, pad_token_id, sp))
df['tp_queries'] = df['query'].apply(lambda x: encode_and_pad(x, max_length, pad_token_id, tokenizer))   
df['tp_passages'] = df['cleaned_paragraphs'].apply(
    lambda paragraphs: [encode_and_pad(paragraph, max_length, pad_token_id, tokenizer) for paragraph in paragraphs]
)

In [12]:
padded_passages = df['tp_passages']
#print(padded_passages[0][0])

In [13]:
padded_queries = df['tp_queries'] 
number_of_rows = len(padded_queries.index)
print(f"The DataFrame has {number_of_rows} rows.")

The DataFrame has 9650 rows.


In [ ]:
# Flatten the list of lists of tokens
flat_list_of_tokens = [token for sublist in df['tp_passages'].explode() for token in sublist]
flat_list_of_tokens += [token for token in df['tp_queries']]

flat_list_of_tokens = [token for sublist in flat_list_of_tokens for token in sublist]

In [ ]:
#print("Structure of the first document:", padded_passages[0][0])

# Attempt to flatten the first document if it's a list of lists
flat_doc = [item for sublist in padded_passages[0][0] for item in sublist]

# Check the flattened document
print("Flattened document:", flat_doc)

# Try to convert the flattened document to a tensor
try:
    doc_tensor = torch.tensor([flat_doc], dtype=torch.long)
    print("Tensor created successfully.")
except Exception as e:
    print("An error occurred while creating the tensor:", e)

In [ ]:
print(f"Total number of elements in flat_list_of_tokens: {len(flat_list_of_tokens)}")

# Check for any sublists
for i, item in enumerate(flat_list_of_tokens):
    if isinstance(item, list):
        print(f"Found a sublist at index {i}: {item}")
        break
else:
    print("The list is flat.")

In [ ]:
import numpy as np
flat_list = [item for sublist in flat_list_of_tokens for item in (sublist if isinstance(sublist, list) else [sublist])]

# Now you can use np.unique on the flattened list
unique_tokens, counts = np.unique(flat_list, return_counts=True)

#sum(counts)
unique_tokens[:9560]

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import torch.nn.functional as F

class SiameseLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(SiameseLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.cosine_similarity = nn.CosineSimilarity(dim=1, eps=1e-6)

    def forward(self, query, doc):
        query_embedding = self.embedding(query)
        doc_embedding = self.embedding(doc)

        # Get the outputs and the last hidden state from the LSTM
        query_output, (query_hidden, _) = self.lstm(query_embedding)
        doc_output, (doc_hidden, _) = self.lstm(doc_embedding)

        # Since the LSTM is bidirectional, we need to concatenate the final forward
        # and backward hidden states
        query_hidden = torch.cat((query_hidden[-2,:,:], query_hidden[-1,:,:]), dim = 1)
        doc_hidden = torch.cat((doc_hidden[-2,:,:], doc_hidden[-1,:,:]), dim = 1)

        # Calculate cosine similarity on the final hidden state vectors
        similarity = self.cosine_similarity(query_hidden, doc_hidden)
        return similarity

    def contrastive_loss(self, positive_similarity, negative_similarities, margin=0.5):
        losses = [F.relu(margin - positive_similarity + neg_sim) for neg_sim in negative_similarities]
        loss = sum(losses) / len(losses)
        return loss.mean()


In [15]:
import wandb
wandb.init(project='my_siamese_project', entity='pelumia23')

wandb: Currently logged in as: pelumia23. Use `wandb login --relogin` to force relogin


In [16]:
filtered_queries_n = []
filtered_passages_n = []
filtered_labels = []
filtered_cleaned_paragraphs = []
filtered_query = []
for i, encoded_query in enumerate(padded_queries):
    relevant_docs_indices = df['cleaned_passages_with_labels'][i]
    if relevant_docs_indices:  # Only keep queries with relevant documents
        filtered_queries_n.append(encoded_query)
        filtered_passages_n.append(padded_passages[i])
        filtered_query.append(df['query'][i])
        filtered_cleaned_paragraphs.append(df['cleaned_paragraphs'][i])
        filtered_labels.append(relevant_docs_indices)
print(filtered_query[2])
print(filtered_cleaned_paragraphs[2][2])



what is a apothem
Apothem of a Regular Polygon. Definition: A line segment from the center of a regular polygon to the midpoint of a side. Try this Adjust the polygon below by dragging any orange dot, or alter the number of sides. Note the behavior of the apothem line shown in blue. The word apothem can refer to the line itself, or the length of that line. So you can correctly say 'draw the apothem' and 'the apothem is 4cm'. Each formula below shows how to find the length of the apothem of a regular polygon. Use the formula that uses the facts you are given to start.


In [17]:
vocab_size = 3000
embedding_dim = 50
hidden_dim = 64
num_layers = 2
siamese_model = SiameseLSTM(vocab_size, embedding_dim, hidden_dim, num_layers)
optimizer = optim.Adam(siamese_model.parameters(), lr=0.01)
num_epochs = 3

for epoch in range(num_epochs):
    siamese_model.train()  # Set the model to training mode
    total_loss = 0.0
    total_iterations = 0

    for i, encoded_query in enumerate(filtered_queries_n[:500]):
        query_tensor = torch.LongTensor(encoded_query)
        relevant_doc_index = filtered_labels[i][0]  # Get the index of the relevant document
        positive_doc_tensor = torch.LongTensor(filtered_passages_n[i][relevant_doc_index])  # Index directly with the relevant doc index

        negative_indices = random.sample([x for x in range(len(filtered_queries_n)) if x != i], 4)
        negative_indices.append(i)  # Ensure one negative sample comes from the same query index

        n_similarities = []
        for neg_idx in negative_indices:
                if neg_idx == i:
                    # Avoid choosing the relevant document as a negative sample
                    non_relevant_docs = [idx for idx in range(len(filtered_passages_n[neg_idx])) if idx != relevant_doc_index]
                    if not non_relevant_docs:
                        continue  # Skip if no non-relevant docs are available
                    doc_idx = random.choice(non_relevant_docs)
                else:
                    doc_idx = random.choice(range(len(filtered_passages_n[neg_idx])))

                negative_doc_tensor = torch.LongTensor(filtered_passages_n[neg_idx][doc_idx])

                # Calculate similarity
                n_similarity = siamese_model(query_tensor, negative_doc_tensor)
                n_similarities.append(n_similarity)

        # Calculate the positive similarity
        positive_similarity = siamese_model(query_tensor, positive_doc_tensor)

        optimizer.zero_grad()
        loss = siamese_model.contrastive_loss(positive_similarity, n_similarities)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        wandb.log({'batch_loss': loss.item(), 'epoch': epoch})
        total_iterations += 1
        
mean_loss = total_loss / total_iterations if total_iterations > 0 else 0
print(f"Epoch {epoch + 1}: Mean Loss = {mean_loss}")


Epoch 3: Mean Loss = 0.42928335806261747


In [22]:
def rank_documents_for_query(model, query_index, queries, docs):
    """
    Rank documents based on their similarity to a query using a Siamese LSTM model.

    Parameters:
    - model: A trained SiameseLSTM model.
    - query_index: The index of the query in the queries list.
    - queries: A list of queries, each represented as a tensor.
    - docs: A list of documents, each represented as a tensor.

    Returns:
    - A list of tuples (doc_index, similarity_score) sorted by similarity score in descending order.
    """
    model.eval()  # Put the model in evaluation mode
    query_tensor = torch.LongTensor(queries[query_index])  # Convert to tensor and add batch dimension

    similarities = []
    with torch.no_grad():  # No need to track gradients for evaluation
        for i, doc in enumerate(docs):
            doc_tensor = torch.LongTensor(doc)  # Convert to tensor and add batch dimension
            similarity_tensor = model(query_tensor, doc_tensor)
            # Ensure the tensor is a single value tensor
            if similarity_tensor.numel() == 1:
                similarity = similarity_tensor.item()
            else:
                # If the similarity tensor has more than one value, take the mean
                similarity = similarity_tensor.mean().item()
            similarities.append((i, similarity))

    # Sort the documents by similarity score in descending order
    ranked_docs_with_scores = sorted(similarities, key=lambda x: x[1], reverse=True)
    return ranked_docs_with_scores

# ... rest of your code ...

# When printing the results:


# Assuming `siamese_model` is your trained SiameseLSTM model instance,
# `query_tensor` is the tensor for your single query,
# and `doc_tensors` is a list of tensors for the documents to rank
query_index = 1200
ranked_docs_with_scores = rank_documents_for_query(siamese_model, query_index, filtered_queries_n, filtered_passages_n[query_index])


def truncate_text_by_words(text, max_words=20):
    words = text.split()
    return ' '.join(words[:max_words]) + ('...' if len(words) > max_words else '')

# Example usage with word truncation:
#print(f"Query: {truncate_text_by_words(filtered_query[query_index])}")
#for doc_index, score in ranked_docs_with_scores:
    #doc_text = filtered_cleaned_paragraphs[query_index][doc_index]
    #print(f"Document {doc_index}: Score {score:.4f} | Text: {truncate_text_by_words(doc_text)}")


In [23]:
print(f"Query: {truncate_text_by_words(filtered_query[query_index])}")
for doc_index, score in ranked_docs_with_scores:
    doc_text = filtered_cleaned_paragraphs[query_index][doc_index]
    # Format the score as float
    print(f"Document {doc_index}: Score {float(score):.4f} | Text: {truncate_text_by_words(doc_text)}")

Query: what is a cheetah's natural habitat
Document 4: Score 0.9345 | Text: The cheetah (Acinonyx jubatus) is a big cat in the subfamily Felinae that inhabits most of Africa and parts of...
Document 8: Score 0.9211 | Text: The cheetah runs down its prey and therefore needs open spaces across which to do this. they enjoy eating frogs...
Document 1: Score 0.9030 | Text: Here are some of the most interesting and amazing cheetah facts for kids. The cheetah (Acinonyx jubatus) is a feline...
Document 7: Score 0.8808 | Text: Cheetahs are well studied in their natural habitat; however, studying cheetah biology in the wild remains difficult. Nuances of cheetah...
Document 5: Score 0.8798 | Text: Cheetahs are carnivores, meaning their primary food is meat. The predators stick to smaller prey, such as gazelles, hares, young...
Document 3: Score 0.8600 | Text: Cheetahs may be fast but the question that wildlife biologists ask is, are they fast enough to outrun extinction? Today...
Document 0: Scor

In [ ]:
for i, doc in enumerate(filtered_cleaned_paragraphs[query_index]):
    print(doc)